# 1 Common data problems

Overcoming some of the most common dirty data problems. Converting data types, apply range constraints to remove future data points, and remove duplicated data points to avoid double-counting.

## Data types constraints

### Numeric data or ... ?

Exploring bicycle ride sharing data in San Francisco called `ride_sharing`. It contains information on the start and end stations, the trip duration, and some user information for a bike sharing service.

The `user_type` column contains information on whether a user is taking a free ride and takes on the following values:

* `1` for free riders.
* `2` for pay per ride.
* `3` for monthly subscribers.

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ride_sharing = pd.read_csv("../data/ride_sharing.csv")
ride_sharing.head()

,ride_id,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender,tire_sizes,ride_date
0,0,11,16,Steuart St at Market St,93,4th St at Mission Bay Blvd S,5504,Subscriber,1988,Male,27,2018-03-04
1,1,8,3,Powell St BART Station (Market St at 4th St),93,4th St at Mission Bay Blvd S,2915,Subscriber,1988,Male,27,2017-03-27
2,2,11,15,San Francisco Ferry Building (Harry Bridges Pl...,67,San Francisco Caltrain Station 2 (Townsend St...,5340,Customer,1988,Male,26,2019-06-30
3,3,7,21,Montgomery St BART Station (Market St at 2nd St),50,2nd St at Townsend St,746,Subscriber,1969,Male,27,2018-11-16
4,4,11,81,Berry St at 4th St,21,Montgomery St BART Station (Market St at 2nd St),5477,Subscriber,1986,Male,26,2017-11-01


In [3]:
ride_sharing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ride_id          78 non-null     int64 
 1   duration         78 non-null     int64 
 2   station_A_id     78 non-null     int64 
 3   station_A_name   78 non-null     object
 4   station_B_id     78 non-null     int64 
 5   station_B_name   78 non-null     object
 6   bike_id          78 non-null     int64 
 7   user_type        78 non-null     object
 8   user_birth_year  78 non-null     int64 
 9   user_gender      78 non-null     object
 10  tire_sizes       78 non-null     int64 
 11  ride_date        78 non-null     object
dtypes: int64(7), object(5)
memory usage: 7.4+ KB


In [4]:
ride_sharing.user_type.describe()

count             78
unique             2
top       Subscriber
freq              71
Name: user_type, dtype: object

By looking at the summary statistics - they don't really seem to offer much description on how users are distributed along their purchase type, The `user_type` column has an finite set of possible values that represent groupings of data, it should be converted to `category`.

In [5]:
ride_sharing["user_type_cat"] = ride_sharing.user_type.astype("category")
assert ride_sharing.user_type_cat.dtype == "category"

In [6]:
ride_sharing.user_type_cat.describe()

count             78
unique             2
top       Subscriber
freq              71
Name: user_type_cat, dtype: object

it seems that most users are pay per ride users!

### Summing strings and concatenating numbers

Converting the string column `duration` to the type `int`.

In [7]:
ride_sharing.dtypes

ride_id               int64
duration              int64
station_A_id          int64
station_A_name       object
station_B_id          int64
station_B_name       object
bike_id               int64
user_type            object
user_birth_year       int64
user_gender          object
tire_sizes            int64
ride_date            object
user_type_cat      category
dtype: object

## Data Range Constraints

### Tire size contraints

Working with the tire_sizes column which contains data on each bike's tire size. Bicycle tire sizes could be either `26″`, `27″` or `29″` and are here correctly stored as a categorical value. In an effort to cut maintenance costs, the ride sharing provider decided to set the maximum tire size to be `27″`. Let's make sure the `tire_sizes` column has the correct range by first converting it to an integer, then setting and testing the new upper limit of `27″` for tire sizes.

In [8]:
# Convert tire_sizes to integer
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')

# Set all values above 27 to 27
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27

# Reconvert tire_sizes back to categorical
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')

# Print tire size description
ride_sharing['tire_sizes'].describe()

count     78
unique     2
top       27
freq      45
Name: tire_sizes, dtype: int64

### Back to the future

A bug was discovered which was relaying rides taken today as taken next year. To fix this, we will find all instances of the `ride_date` column that occur anytime in the future, and set the maximum possible value of this column to today's date. Before doing so, we need to convert `ride_date` to a datetime object.

In [9]:
# Convert ride_date to datetime
ride_sharing['ride_date'] = pd.to_datetime(ride_sharing['ride_date'])

# Save today's date
today = pd.to_datetime('today')

# Set all in the future to today's date
ride_sharing.loc[ride_sharing['ride_date'] > today, 'ride_date'] = today

# Print maximum of ride_date column
ride_sharing['ride_date'].max()

Timestamp('2020-01-17 00:00:00')

## Uniqueness Constraints

### Finding duplicates

The number of rides taken has increased by `20%` overnight, leading us to think there might be both complete and incomplete duplicates in the `ride_sharing` DataFrame. Let's confirm this suspicion by finding those duplicates. 

In [10]:
# Find duplicates
duplicates = ride_sharing.duplicated('ride_id', keep=False)

# Sort your duplicated rides
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print relevant columns of duplicated_rides
print(duplicated_rides[['ride_id','duration','user_birth_year']])

    ride_id  duration  user_birth_year
22       33        10             1979
39       33         2             1979
53       55         9             1985
65       55         9             1985
74       71        11             1997
75       71        11             1997
76       89         9             1986
77       89         9             2060


Notice that rides 33 and 89 are incomplete duplicates, whereas the remaining are complete.

### Treating duplicates

Let's treat those duplicated rows by first dropping complete duplicates, and then merging the incomplete duplicate rows into one while keeping the average `duration`, and the minimum `user_birth_year` for each set of incomplete duplicate rows.

In [11]:
# Drop complete duplicates from ride_sharing
ride_dup = ride_sharing.drop_duplicates()

# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

# 2 Text and categorical data problems

Categorical and text data can often be some of the messiest parts of a dataset due to their unstructured nature. In this section, we wil fix whitespace and capitalization inconsistencies in category labels, collapse multiple categories into one, and reformat strings for consistency.

## Membership constraints

We will be working with the `airlines` DataFrame which contains survey responses on the San Francisco Airport from airline customers. The DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding cleanliness, safety, and satisfaction. Another DataFrame named `categories` was created, containing all correct possible values for the survey columns. We will use both of these DataFrames to find survey answers with inconsistent values, and drop them, effectively performing an outer and inner join on both these DataFrames

In [12]:
airlines = pd.read_csv('../data/airlines.csv', index_col=0)
categories = pd.read_csv('../data/categories.csv')

In [13]:
airlines.head()

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,Asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,Canada/Mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied
2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,West US,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral
3,1157,Dr. Christine Nicholson,Tuesday,SOUTHWEST,LOS ANGELES,West US,Hub,Gates 20-39,2018-12-31,190.0,Clean,Very safe,Somewhat satsified
4,2992,Regina Clements,Wednesday,AMERICAN,MIAMI,East US,Hub,Gates 50-59,2018-12-31,559.0,Somewhat clean,Very safe,Somewhat satsified


In [14]:
categories

,cleanliness,safety,satisfaction
0,Clean,Neutral,Very satisfied
1,Average,Very safe,Neutral
2,Somewhat clean,Somewhat safe,Somewhat satisfied
3,Somewhat dirty,Very unsafe,Somewhat unsatisfied
4,Dirty,Somewhat unsafe,Very unsatisfied


In [15]:
# Print unique values of survey columns in airlines
print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ', airlines.safety.unique(), "\n")
print('Satisfaction: ', airlines.satisfaction.unique(), "\n")

Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'] 



In [16]:
# Find the cleanliness category in airlines not in categories
cat_clean = set(airlines.cleanliness).difference(categories.cleanliness)

# Find rows with that category
cat_clean_rows = ~(airlines['cleanliness'].isin(categories.cleanliness))

# Print rows with inconsistent category
airlines[cat_clean_rows]

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction


In [17]:
airlines[~cat_clean_rows]

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,Asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,Canada/Mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied
2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,West US,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral
3,1157,Dr. Christine Nicholson,Tuesday,SOUTHWEST,LOS ANGELES,West US,Hub,Gates 20-39,2018-12-31,190.0,Clean,Very safe,Somewhat satsified
4,2992,Regina Clements,Wednesday,AMERICAN,MIAMI,East US,Hub,Gates 50-59,2018-12-31,559.0,Somewhat clean,Very safe,Somewhat satsified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,819,Ms. Vanna Rivera,Sunday,ALASKA,PORTLAND,West US,Medium,Gates 50-59,2018-12-31,80.0,Somewhat clean,Neutral,Neutral
196,2924,Miss Venus Lowe,Friday,TURKISH AIRLINES,ISTANBUL,Middle East,Hub,Gates 91-102,2018-12-31,205.0,Somewhat clean,Somewhat safe,Somewhat satsified
197,2245,Amethyst Nieves,Thursday,SOUTHWEST,PHOENIX,West US,Hub,Gates 20-39,2018-12-31,140.0,Average,Somewhat safe,Somewhat satsified
198,238,Miss Vivian Foreman,Wednesday,AMERICAN,LOS ANGELES,West US,Hub,Gates 50-59,2018-12-31,115.0,Somewhat clean,Very safe,Somewhat satsified


## Categorical variables

### Inconsistent categories

We will examine two categorical columns from this DataFrame, `dest_region` and `dest_size` respectively, assess how to address them and make sure that they are cleaned and ready for analysis.

In [18]:
# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium']


#### Observation
1. The `dest_region` column has inconsistent values due to capitalization and has one value that needs to be remapped.
2. The `dest_size` column has only inconsistent values due to leading and trailing spaces.


In [19]:
# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

# Lower dest_region column and then replace "eur" with "europe"
airlines['dest_region'] = airlines['dest_region'].str.lower() 
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

# Remove white spaces from `dest_size`
airlines['dest_size'] = airlines['dest_size'].str.strip()

# Verify changes have been effected
print(airlines.dest_region.unique())
print(airlines.dest_size.unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium']
['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']
['Hub' 'Small' 'Medium' 'Large']


### Remapping categories

To better understand survey respondents from `airlines`, we want to find out if there is a relationship between certain responses and the day of the week and wait time at the gate. The airlines DataFrame contains the `day` and `wait_min` columns, which are categorical and numerical respectively. The `day` column contains the exact day a flight took place, and `wait_min` contains the amount of minutes it took travelers to wait at the gate. To make our analysis easier, let us create two new categorical variables:

* `wait_type`: `'short'` for 0-60 min, `'medium'` for 60-180 and `long` for 180+
* `day_week`: `'weekday'` if day is in the `weekday`, `'weekend'` if day is in the weekend.

In [20]:
# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines.wait_min, bins = label_ranges, 
                                labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)
airlines.head()

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,day_week
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,weekday
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,weekday
2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium,weekday
3,1157,Dr. Christine Nicholson,Tuesday,SOUTHWEST,LOS ANGELES,west us,Hub,Gates 20-39,2018-12-31,190.0,Clean,Very safe,Somewhat satsified,long,weekday
4,2992,Regina Clements,Wednesday,AMERICAN,MIAMI,east us,Hub,Gates 50-59,2018-12-31,559.0,Somewhat clean,Very safe,Somewhat satsified,long,weekday


## Cleaning text data

### Removing titles and taking names

While collecting survey respondent metadata in the airlines DataFrame, the full name of respondents was saved in the `full_name` column. However upon closer inspection, we found that a lot of the different names are prefixed by honorifics such as `"Dr."`, `"Mr."`, `"Ms."` and `"Miss"`. Our ultimate objective is to create two new columns named `first_name` and `last_name`, containing the first and last names of respondents respectively. Before doing so however, we need to remove honorifics.

In [21]:
# Replace "Dr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")

# Replace "Mr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")

# Replace "Miss" with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Miss","")

# Replace "Ms." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")

# Assert that full_name has no honorifics
assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

 By normalizing full names this way, you can now easily split them into first names and last names!
 
 ### Keeping it descriptive
 
 To further understand travelers' experiences in the San Francisco Airport, the quality assurance department sent out a qualitative questionnaire to all travelers who gave the airport the worst score on all possible categories. The objective behind this questionnaire is to identify common patterns in what travelers are saying about the airport. Their response is stored in the `survey_response` column. Upon a closer look, we realized a few of the answers gave the shortest possible character amount without much substance. We will isolate the responses with a character count higher than 40 , and make sure your new DataFrame contains responses with 40 characters or more using an `assert` statement.

In [22]:
airlines = pd.read_csv('../data/survey_response.csv', index_col=0)
# Store length of each row in survey_response column
resp_length = airlines.survey_response.str.len()

# Find rows in airlines where resp_length > 40
airlines_survey = airlines[resp_length > 40]

# Assert minimum survey_response length is > 40
assert airlines_survey.survey_response.str.len().min() > 40

# Print new survey_response column
print(airlines_survey['survey_response'])

18    The airport personnell forgot to alert us of d...
19    The food in the airport was really really expe...
20    One of the other travelers was really loud and...
21    I don't remember answering the survey with the...
22    The airport personnel kept ignoring my request...
23    The chair I sat in was extremely uncomfortable...
24    I wish you were more like other airports, the ...
25    I was really unsatisfied with the wait times b...
27    The flight was okay, but I didn't really like ...
28    We were really slowed down by security measure...
29    There was a spill on the aisle next to the bat...
30    I felt very unsatisfied by how long the flight...
Name: survey_response, dtype: object
